In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
import pickle
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

In [ ]:
nn_tuned_parameters = [{'hidden_layer_sizes': [(,), (50,), (50, 10), (50, 10, 10)],
                        'alpha':[0.0001,0.001,0.01,0.1,1,10], 'max_iter': range(50,200,50)}]

In [ ]:
data = pd.read_csv('data/processed/mushrooms_pca.csv')
y = data['class']
X = data.drop('class', axis=1)

X_train, X_test, y_train, y_test = \
        train_test_split(X, y.ravel(), test_size=.1, random_state=42)

In [ ]:
clf = MLPClassifier()

grid_search_nn = GridSearchCV(clf, nn_tuned_parameters, scoring='f1',
                           cv=10, return_train_score=True, n_jobs=3)
grid_search_nn.fit(X_train, y_train)


In [ ]:
y_true, y_pred = y_test, grid_search_nn.predict(X_test)
print(grid_search_nn.best_params_)
report = classification_report(y_true, y_pred)
print(report)

In [ ]:
res=pd.DataFrame(grid_search_nn.cv_results_).sort_values('rank_test_score').reset_index(drop=True)
res = res[['param_hidden_layer_sizes', 'param_alpha', 'param_max_iter', 'mean_test_score', 'mean_fit_time']]
print(res.head(n=8).to_latex())
pickle.dump(grid_search_nn, open("models/nn.p", "wb" ) )

In [12]:
perc = Perceptron(tol=1e-3, random_state=0)
perc.fit(X_train, y_train)

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
      fit_intercept=True, max_iter=None, n_iter=None, n_iter_no_change=5,
      n_jobs=None, penalty=None, random_state=0, shuffle=True, tol=0.001,
      validation_fraction=0.1, verbose=0, warm_start=False)

In [24]:
scores = cross_val_score(perc, X_train, y_train, cv=10, scoring='f1_macro')

perc_f1=scores.mean()

In [2]:
nn_tuned_parameters = [{'hidden_layer_sizes': [(50,), (50, 10), (50, 10, 10), (50, 10, 10, 10)]}]

In [4]:
clf = MLPClassifier(alpha=0.1)

grid_search_nn = GridSearchCV(clf, nn_tuned_parameters, scoring='f1',
                           cv=10, return_train_score=True, n_jobs=3)
grid_search_nn.fit(X_train, y_train)


C:\Users\user\Desktop\kto_ma_szkole_ten_ma_lzej\Erasm\mushrooms\venv\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=MLPClassifier(activation='relu', alpha=0.1, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False),
       fit_params=None, iid='warn', n_jobs=3,
       param_grid=[{'hidden_layer_sizes': [(50,), (50, 10), (50, 10, 10), (50, 10, 10, 10)]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='f1', verbose=0)

In [6]:
res=pd.DataFrame(grid_search_nn.cv_results_).sort_values('rank_test_score').reset_index(drop=True)
res = res[['param_hidden_layer_sizes', 'mean_test_score', 'mean_fit_time']]
print(res.head(n=8).to_latex())


\begin{tabular}{llrr}
\toprule
{} & param\_hidden\_layer\_sizes &  mean\_test\_score &  mean\_fit\_time \\
\midrule
0 &                    (50,) &         0.903014 &      14.816777 \\
1 &                 (50, 10) &         0.891997 &      11.748389 \\
2 &         (50, 10, 10, 10) &         0.891776 &       9.132103 \\
3 &             (50, 10, 10) &         0.887333 &       9.098660 \\
\bottomrule
\end{tabular}



In [49]:
f1scores = np.array(res['mean_test_score'])
plt.plot(list(range(0,len(f1scores)+1)), np.insert(f1scores,0, perc_f1))
plt.xlabel('Number of hidden layers')
plt.ylabel('f1 score')
plt.title('MLP classifier performance')
plt.show()